### 이미지 파일 확인

In [1]:
import os
file_path = "/Users/nayoungmin/Desktop/Capstone/image_yeju"
file_names = [f for f in os.listdir(file_path) if os.path.isdir(os.path.join(file_path, f))]

file_Path = []
# 장소별 데이터 수
for place in file_names:
    PATH = os.path.join(file_path, place)
    print(str(place), ":", str(len(os.listdir(PATH))))
    file_Path.append(PATH)

print(file_Path)


벚꽃 : 442
놀이공원 : 348
타워 : 318
유람선 : 366
케이블카 : 307
빌딩 : 454
노을 : 393
번화가 : 335
돌담길 : 704
궁 : 363
철로 : 434
자연 : 705
아쿠아리움 : 334
피크닉 : 410
아이스링크 : 82
단풍 : 341
전통문화 : 301
야경 : 431
도심하천 : 791
극장 : 865
워터파크 : 320
['/Users/nayoungmin/Desktop/Capstone/image_yeju/벚꽃', '/Users/nayoungmin/Desktop/Capstone/image_yeju/놀이공원', '/Users/nayoungmin/Desktop/Capstone/image_yeju/타워', '/Users/nayoungmin/Desktop/Capstone/image_yeju/유람선', '/Users/nayoungmin/Desktop/Capstone/image_yeju/케이블카', '/Users/nayoungmin/Desktop/Capstone/image_yeju/빌딩', '/Users/nayoungmin/Desktop/Capstone/image_yeju/노을', '/Users/nayoungmin/Desktop/Capstone/image_yeju/번화가', '/Users/nayoungmin/Desktop/Capstone/image_yeju/돌담길', '/Users/nayoungmin/Desktop/Capstone/image_yeju/궁', '/Users/nayoungmin/Desktop/Capstone/image_yeju/철로', '/Users/nayoungmin/Desktop/Capstone/image_yeju/자연', '/Users/nayoungmin/Des

## 모듈 불러오기

In [2]:
import tensorflow as tf
import splitfolders   # 입력된 비율을 기준으로 데이터셋을 분할하는 함수
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import AveragePooling2D
from keras.layers  import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
from keras.metrics import AUC
from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions

## GPU 사용

### 모듈 불러오기

### GPU 확인

In [3]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except:
        print("Cannot set memory growth on device")
    else:
        print("GPU memory growth set to True")
else:
    print("Not enough GPU hardware devices available")


print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

GPU memory growth set to True
Num GPUs Available:  1


In [4]:
import splitfolders   # 입력된 비율을 기준으로 데이터셋을 분할하는 함수
input_folder = "/Users/nayoungmin/Desktop/Capstone/image_yeju"
output = "/Users/nayoungmin/Desktop/Capstone/splited_image_is" # where you want the split datasets saved. one will be created if it does not exist or none is set

splitfolders.ratio(input_folder, output=output, seed=777, ratio=(0.7, 0.3)) # train,test를 8:2 split 후, train,val을 9:1로 split

In [5]:
import os

file_path = '/Users/nayoungmin/Desktop/Capstone/splited_image_is'
file_names = [f for f in os.listdir(file_path) if not f.startswith('.')]  # 숨김파일 제외

for data in file_names:
    PATH = os.path.join(file_path,data)
    plan_file = [f for f in os.listdir(PATH) if not f.startswith('.')]  # 숨김파일 제외
    print("[",str(data),"data ]")
    print()
    for plan in plan_file:
        PATH_ = os.path.join(PATH,plan)
        print(str(plan)," : ",str(len(os.listdir(PATH_))))
    print()

[ train data ]

벚꽃  :  308
놀이공원  :  243
타워  :  221
유람선  :  256
케이블카  :  214
빌딩  :  317
노을  :  275
번화가  :  234
돌담길  :  492
궁  :  254
철로  :  303
자연  :  492
아쿠아리움  :  233
피크닉  :  286
아이스링크  :  57
단풍  :  238
전통문화  :  210
야경  :  301
도심하천  :  553
극장  :  604
워터파크  :  224

[ val data ]

벚꽃  :  133
놀이공원  :  105
타워  :  96
유람선  :  110
케이블카  :  93
빌딩  :  136
노을  :  118
번화가  :  101
돌담길  :  211
궁  :  109
철로  :  130
자연  :  212
아쿠아리움  :  101
피크닉  :  123
아이스링크  :  25
단풍  :  103
전통문화  :  91
야경  :  130
도심하천  :  237
극장  :  260
워터파크  :  96



In [7]:
from keras.preprocessing.image import ImageDataGenerator

TRAIN_PATH = '/Users/nayoungmin/Desktop/Capstone/splited_image_is/train'
VAL_PATH = '/Users/nayoungmin/Desktop/Capstone/splited_image_is/val'


MODEL_PATH = '/Users/nayoungmin/Desktop/Capstone/Code'

BATCH_SIZE = 50 #배치사이즈 원래 50
IMG_HEIGHT = 224  
IMG_WIDTH = 224 
N_EPOCH = 40
LR = 0.05
nb_classes = 82

trainGen = ImageDataGenerator(
    rescale=1./255,        # 값을 0과 1 사이로 변경
    rotation_range=30,      # 무작위 회전각도 30도 이내
    shear_range=0.2,         # 층밀리기 강도 20% (정사각형 -> 평행사변형)
    zoom_range=0.15,            # 무작위 줌 범위 20%
    horizontal_flip=True,    # 무작위로 가로로 뒤짚는다.
    vertical_flip=True
)
valGen = ImageDataGenerator(
    rescale=1./255,       
    rotation_range=30,   
    shear_range=0.2,      
    zoom_range=0.15,            
    horizontal_flip=True,
    vertical_flip=True
)

# flow_from_directory() : 각 이미지마다 디렉토리 이름을 레이블로 사용
train_generator = trainGen.flow_from_directory(      
   TRAIN_PATH,
   class_mode="categorical",
   target_size=(IMG_HEIGHT, IMG_WIDTH),
   shuffle=True,
   batch_size=BATCH_SIZE)

# initialize the validation generator
# flow_from_directory() : 각 이미지마다 디렉토리 이름을 레이블로 사용
validation_generator = valGen.flow_from_directory(  
   VAL_PATH,
   class_mode="categorical",
   target_size=(IMG_HEIGHT, IMG_WIDTH),
   shuffle=True,
   batch_size=BATCH_SIZE)


Found 6315 images belonging to 21 classes.
Found 2720 images belonging to 21 classes.


### base_model

In [72]:
from keras.layers import AveragePooling2D
from keras.layers  import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

from keras.metrics import AUC

from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions


# ResNet50V2 모델을 불러오고, 그 모델의 출력을 활용하여 새로운 fully connected layer를 추가
resnet = ResNet50V2(include_top=False, weights='imagenet', input_shape = (224,224,3))

x = resnet.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x =  Dense(512, activation='relu', input_dim= (224,224,3))(x)
x = BatchNormalization()(x)
x =  Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x =  Dense(21, activation='softmax')(x)  # 여기에서 Dense를 class 수에 맞게 설정해야 함.

model = Model(inputs=resnet.input, outputs=x)

# 기존 학습된 layer들을 freazing(새로운 layer들만 학습하도록)
for layer in resnet.layers:
   layer.trainable = False

# define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:
        lr = lr * 0.1
    return lr

# compile the model with the learning rate scheduler
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy",AUC(multi_label=True,num_labels=21,name='AUC')])

# train the model
H = model.fit_generator(
    generator = train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)]
)

#model.save('my_model.h5')

/var/folders/0q/f448gzfd2rd9vk0q5h6wlvn80000gn/T/ipykernel_2564/228766123.py:56: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/10


KeyboardInterrupt: 

### my_model_parameter_1

In [11]:
from keras.layers import AveragePooling2D
from keras.layers  import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

from keras.metrics import AUC

from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions


# ResNet50V2 모델을 불러오고, 그 모델의 출력을 활용하여 새로운 fully connected layer를 추가
resnet = ResNet50V2(include_top=False, weights='imagenet', input_shape = (224,224,3))

x = resnet.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x =  Dense(512, activation='relu', input_dim= (224,224,3))(x)
x = BatchNormalization()(x)
x =  Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x =  Dense(21, activation='softmax')(x)  # 여기에서 Dense를 class 수에 맞게 설정해야 함.

model = Model(inputs=resnet.input, outputs=x)

# 기존 학습된 layer들을 freazing(새로운 layer들만 학습하도록)
for layer in resnet.layers:
   layer.trainable = True

# define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch > 0 and epoch % 10 == 0:
        lr = lr * 0.05
    return lr

# compile the model with the learning rate scheduler
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy",AUC(multi_label=True,num_labels=21,name='AUC')])

# train the model
H = model.fit_generator(
    generator = train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)]
)

#model.save('my_model.h5')

/var/folders/0q/f448gzfd2rd9vk0q5h6wlvn80000gn/T/ipykernel_1800/1010919424.py:56: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/30
96/96 [==============================] - 100s 946ms/step - loss: 1.6903 - accuracy: 0.5270 - AUC: 0.8947 - val_loss: 6.7268 - val_accuracy: 0.1402 - val_AUC: 0.6779 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/30
96/96 [==============================] - 89s 919ms/step - loss: 0.9028 - accuracy: 0.7369 - AUC: 0.9651 - val_loss: 2.0106 - val_accuracy: 0.5353 - val_AUC: 0.8871 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/30
96/96 [==============================] - 89s 925ms/step - loss: 0.6789 - accuracy: 0.7959 - AUC: 0.9795 - val_loss: 1.4864 - val_accuracy: 0.6522 - val_AUC: 0.9271 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/30
96/96 [==============================] - 88s 917ms/step - loss: 0.5745 - accuracy: 0.8299 - AUC: 0.9842 

In [13]:
model.save('my_model_parameter_1.h5')

### my_model_lr

In [8]:
from keras.layers import AveragePooling2D
from keras.layers  import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.layers import MaxPooling2D
from keras.layers import BatchNormalization
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.models import Sequential
import tensorflow as tf
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse

from keras.metrics import AUC

from keras.applications.resnet_v2 import ResNet50V2, ResNet101V2, ResNet152V2, preprocess_input, decode_predictions
from keras.applications.resnet import ResNet50, preprocess_input, decode_predictions


# ResNet50V2 모델을 불러오고, 그 모델의 출력을 활용하여 새로운 fully connected layer를 추가
resnet = ResNet50V2(include_top=False, weights='imagenet', input_shape = (224,224,3))

x = resnet.output
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Flatten()(x)
x = Dropout(0.5)(x)
x =  Dense(512, activation='relu', input_dim= (224,224,3))(x)
x = BatchNormalization()(x)
x =  Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x =  Dense(21, activation='softmax')(x)  # 여기에서 Dense를 class 수에 맞게 설정해야 함.

model = Model(inputs=resnet.input, outputs=x)

# 기존 학습된 layer들을 freazing(새로운 layer들만 학습하도록)
for layer in resnet.layers:
   layer.trainable = True

# define the learning rate scheduler
def lr_scheduler(epoch, lr):
    if (epoch > 0) and (epoch <= 10) and (epoch % 10 == 0):
        lr = lr * 0.1
    elif (epoch > 10) and (epoch <= 20) and (epoch % 10 == 0):
        lr = lr * 0.05
    elif (epoch > 20) and (epoch <= 30) and (epoch % 10 == 0):
        lr = lr * 0.25
    elif (epoch > 30) and (epoch <= 35) and (epoch % 5 == 0):
        lr = lr * 0.1
    return lr

# compile the model with the learning rate scheduler
opt = Adam()
model.compile(loss="categorical_crossentropy", optimizer=opt,
    metrics=["accuracy",AUC(multi_label=True,num_labels=21,name='AUC')])

# train the model
H = model.fit_generator(
    generator = train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=N_EPOCH,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    callbacks=[tf.keras.callbacks.LearningRateScheduler(lr_scheduler, verbose=1)]
)

#model.save('my_model.h5')

/var/folders/0q/f448gzfd2rd9vk0q5h6wlvn80000gn/T/ipykernel_1679/1207415666.py:62: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  H = model.fit_generator(



Epoch 1: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 1/40


2023-04-17 11:14:37.990535: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


126/126 [==============================] - 93s 695ms/step - loss: 1.9715 - accuracy: 0.4458 - AUC: 0.8606 - val_loss: 2.8627 - val_accuracy: 0.4796 - val_AUC: 0.8570 - lr: 0.0010

Epoch 2: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 2/40
126/126 [==============================] - 86s 682ms/step - loss: 1.1205 - accuracy: 0.6686 - AUC: 0.9505 - val_loss: 2.2909 - val_accuracy: 0.5363 - val_AUC: 0.8862 - lr: 0.0010

Epoch 3: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 3/40
126/126 [==============================] - 85s 677ms/step - loss: 0.8445 - accuracy: 0.7413 - AUC: 0.9701 - val_loss: 1.3986 - val_accuracy: 0.6933 - val_AUC: 0.9331 - lr: 0.0010

Epoch 4: LearningRateScheduler setting learning rate to 0.0010000000474974513.
Epoch 4/40
126/126 [==============================] - 86s 680ms/step - loss: 0.7305 - accuracy: 0.7725 - AUC: 0.9779 - val_loss: 1.4118 - val_accuracy: 0.6163 - val_AUC: 0.9310 - lr: 0.0010

Epoch 5: L

In [9]:
model.save('my_model_lr.h5')

## 이미지 분류
- image_predict 파일 따로 올려놓음

In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator


# 추천 여행지 파일
recommend_csv = pd.read_csv('/Users/nayoungmin/Desktop/Capstone/recommend.csv')


# 저장한 모델 파일 경로
model_path = '/Users/nayoungmin/Desktop/Capstone/Code/my_model_parameter_1.h5'
TRAIN_PATH = '/Users/nayoungmin/Desktop/Capstone/splited_image_is/train'

# 저장한 모델 불러오기
model = load_model(model_path)

# 이미지 클래스 이름 지정해주기
classGen = ImageDataGenerator()

class_generator = classGen.flow_from_directory(
    directory=TRAIN_PATH,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

while(True):
    img_path = input('이미지를 넣어주세요: ') # 예측하고자 하는 이미지 경로
    if img_path == '종료':
        break
    target_size = (224, 224) # 모델이 학습시킨 이미지 사이즈와 동일하게 설정

    # 이미지 전처리
    img = load_img(img_path, target_size=target_size)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = x / 255.0 # 이미지 스케일링

    # 예측
    preds = model.predict(x)

    # 클래스 이름 받는 Dictionary 만들기
    pre_dict = {}

    # 클래스별 확률 값 출력
    class_indices = class_generator.class_indices
    class_names = list(class_indices.keys())
    
    
    for i in range(len(class_names)):
        pre_dict[class_names[i]] = round(preds[0][i]*100, 2)        
    
    sorted_pre_dict = sorted(pre_dict.items(), key=lambda x: x[1], reverse=True)

    for key, value in sorted_pre_dict:
        print(f"{key}: {value}%")

    if max(pre_dict.values()) < 40:
        print('\n\n다른 사진을 업로드 해주세요!\n\n')
        continue
    else:
        max_pre = max(pre_dict, key=pre_dict.get)
        print('카테고리: ',max_pre)
        
        
    input_rec = input('카테고리: ')
    print(recommend_csv.loc[recommend_csv['category']==input_rec, ])


Metal device set to: Apple M1 Pro
Found 6315 images belonging to 21 classes.
이미지를 넣어주세요: /Users/nayoungmin/Desktop/이미지예상/IMG_5959.JPG


2023-04-17 13:00:32.720914: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 1s 662ms/step
워터파크: 97.99%
아쿠아리움: 0.82%
놀이공원: 0.31%
철로: 0.22%
피크닉: 0.15%
도심하천: 0.12%
케이블카: 0.08%
돌담길: 0.06%
극장: 0.05%
전통문화: 0.05%
벚꽃: 0.04%
궁: 0.03%
자연: 0.02%
노을: 0.01%
단풍: 0.01%
번화가: 0.01%
아이스링크: 0.01%
유람선: 0.01%
빌딩: 0.0%
야경: 0.0%
타워: 0.0%
카테고리:  워터파크
카테고리: 워터파크
   category         name sector
30     워터파크  SeaLaLa워터파크   워터파크
31     워터파크         워터킹덤   워터파크
32     워터파크   난지한공공원물놀이장    수영장
이미지를 넣어주세요: 종료
